In [17]:
import pandas as pd
import warnings
import re
from nltk.corpus import stopwords, names
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
warnings.filterwarnings(action='ignore') 
stops = stopwords.words('english')
stops.append('a')
wnl = WordNetLemmatizer()
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [18]:
test.facts[0]

'The 1984 Bail Reform Act allowed the federal courts to detain an arrestee prior to trial if the government could prove that the individual was potentially dangerous to other people in the community. Prosecutors alleged that Salerno and another person in this case were prominent figures in the La Cosa Nostra crime family.\n'

In [19]:
test = test.drop(1)
test.reset_index(drop=True)

,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
2,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
3,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
4,TEST_0005,Air Line Pilots Association,Miller,"The Air Line Pilots Association (ALPA), a priv..."
...,...,...,...,...
1234,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1235,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1236,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1237,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


In [ ]:
from nltk.corpus import stopwords, names
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

stops = stopwords.words('english')
stops.append('a')
stops.append('s')

wnl = WordNetLemmatizer()

In [ ]:
reg = re.compile('[a-zA-Z]+')
for i in range(len(train)):
    first = reg.findall(train.first_party[i]) 
    second = reg.findall(train.second_party[i])
    first.extend([i.lower() for i in first])
    second.extend([i.lower() for i in second])

    tp = nltk.tag.pos_tag(word_tokenize(train.facts[i]))
    tp = [t[0] for t in tp if (t[1] != "TO") and (t[1] != 'DT') and (t[1] != 'CC') and (t[1] != 'IN')]
    tp = ' '.join(tp)    
    
    fact = ' '.join(reg.findall(tp))

    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <fst> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <scd> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]
    for j in tp:
        if j in stops:
            tp.remove(j)

    fact = ' '.join(tp)  
    tp = word_tokenize(fact)
    fact = ' '.join(tp)
    fact = fact.replace('<','').replace('>','')
                
    train.loc[i,'facts'] = fact
            
for i in range(len(test)):
    first = reg.findall(test.first_party[i])
    second = reg.findall(test.second_party[i])
    first.extend([i.lower() for i in first])
    second.extend([i.lower() for i in second])
    
    tp = nltk.tag.pos_tag(word_tokenize(test.facts[i]))
    tp = [t[0] for t in tp if (t[1] != "TO") and (t[1] != 'DT') and (t[1] != 'CC') and (t[1] != 'IN')]
    tp = ' '.join(tp)        

    fact = ' '.join(reg.findall(tp))
    
#     for s in stops:
#         if s in fact:
#             fact = fact.replace(s,'')
    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <fst> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <scd> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]

    for j in tp:
        if j in stops:
            tp.remove(j)
    
    fact = ' '.join(tp)
    tp = word_tokenize(fact)
    fact = ' '.join(tp)
    fact = fact.replace('<','').replace('>','')
    
    test.loc[i,'facts'] = fact